In [1]:
#pip install --upgrade google-api-python-client

In [2]:
#pip install --upgrade google-auth-oauthlib google-auth-httplib2

In [3]:
import os
import google_auth_oauthlib.flow
import googleapiclient.discovery
import googleapiclient.errors
import pandas as pd

In [4]:
scopes = ["https://www.googleapis.com/auth/youtube.force-ssl"]

In [5]:
os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"
    
api_service_name = "youtube"
api_version = "v3"
client_secrets_file = "client_secret_615288565318-dd7us4i9infmai01fr3n2nlff2cb44f3.apps.googleusercontent.com.json"
    
flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(client_secrets_file, scopes)
credentials = flow.run_console()
youtube = googleapiclient.discovery.build(api_service_name, api_version, credentials=credentials)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=615288565318-dd7us4i9infmai01fr3n2nlff2cb44f3.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.force-ssl&state=lJIIjrGx1Ed8LAaRIGGiKuesvAjhEl&prompt=consent&access_type=offline
Enter the authorization code: 4/1AY0e-g4jv8EgyRzpYNWI-JQs7G650wo-TbFw3We_dfid6W1WbnD34T4-54Q


In [6]:
def simple_search(keywords, num_results):
    request = youtube.search().list(
        q=keywords,
        part="id",
        type="video",
        fields="items/id",
        maxResults=num_results
    )
    response = request.execute()
    
    videos = []
    for search_result in response.get("items",[]):
        videos.append("%s" % (search_result["id"]["videoId"]))
    
    return videos

In [7]:
def get_statistics(video_ids):
    statistics = []
    for vid_id in video_ids:
        request = youtube.videos().list(part="statistics",id=vid_id)
        response = request.execute()
        statistics.append(response)
    
    return statistics

In [8]:
def parse_statistics(video_stats):
    stat_dict = {}
    stat_dict['views'] = []
    stat_dict['likes'] = []
    stat_dict['dislikes'] = []
    stat_dict['comments'] = []
    
    for vid_stat in video_stats:
        try:
            stat_dict['views'].append(int(vid_stat['items'][0]['statistics']['viewCount']))
        except:
            stat_dict['views'].append(0)
        
        try:
            stat_dict['likes'].append(int(vid_stat['items'][0]['statistics']['likeCount']))
        except:
            stat_dict['likes'].append(0)
            
        try:
            stat_dict['dislikes'].append(int(vid_stat['items'][0]['statistics']['dislikeCount']))
        except:
            stat_dict['dislikes'].append(0)
            
        try:
            stat_dict['comments'].append(int(vid_stat['items'][0]['statistics']['commentCount']))
        except:
            stat_dict['comments'].append(0)
    
    return stat_dict

In [9]:
def determine_target (video_views):
    target = []
    for views in video_views:
        if views > 1000000:
            target.append(1)
        else:
            target.append(0)
    return target

In [10]:
num_searches = 10;
music_vids=simple_search("music",num_searches)
sports_vids=simple_search("sports",num_searches)
gaming_vids=simple_search("gaming",num_searches)
movies_vids=simple_search("movies and shows",num_searches)
news_vids=simple_search("news",num_searches)
live_vids=simple_search("live",num_searches)
fashion_vids=simple_search("fashion and beauty",num_searches)
learning_vids=simple_search("learning",num_searches)
vlog_vids=simple_search("vlog",num_searches)
drawing_vids=simple_search("drawing",num_searches)

In [17]:
ids_df = pd.DataFrame(
    data=music_vids+
    sports_vids+
    gaming_vids+
    movies_vids+
    news_vids+
    live_vids+
    fashion_vids+
    learning_vids+
    vlog_vids+
    drawing_vids,
    columns=["Video_IDs"]
)
ids_df

,Video_IDs
0,PuJQhtg1VCM
1,opdIpVmytBM
2,89XPBelvrZk
3,W9iUh23Xrsg
4,sGaTWieIHJs
...,...
95,FkBslQBUVeA
96,5aKJrysT-_M
97,mM2LQDy-e_8
98,6IeaiSVU7Cc


In [18]:
video_stats = get_statistics(ids_df['Video_IDs'])
video_stats

[{'kind': 'youtube#videoListResponse',
  'etag': 'wA0-ZjkOf039fwqjo2IZVnLiLvY',
  'items': [{'kind': 'youtube#video',
    'etag': '1p9XemtA5o9GvL3MY3RASf5kjkU',
    'id': 'PuJQhtg1VCM',
    'statistics': {'viewCount': '20060174',
     'likeCount': '116457',
     'dislikeCount': '7690',
     'favoriteCount': '0',
     'commentCount': '10298'}}],
  'pageInfo': {'totalResults': 1, 'resultsPerPage': 1}},
 {'kind': 'youtube#videoListResponse',
  'etag': 'JE_lQh_ylLdIPct9ype4e3dxZvs',
  'items': [{'kind': 'youtube#video',
    'etag': 'kDDAaZrh5kCfFhWUCwuQsYi5IO0',
    'id': 'opdIpVmytBM',
    'statistics': {'viewCount': '6894',
     'likeCount': '173',
     'dislikeCount': '3',
     'favoriteCount': '0',
     'commentCount': '0'}}],
  'pageInfo': {'totalResults': 1, 'resultsPerPage': 1}},
 {'kind': 'youtube#videoListResponse',
  'etag': 'GAh4SjaMRPJN0TfYtqPgGihdmvI',
  'items': [{'kind': 'youtube#video',
    'etag': 'UFu8AizejI3QSKeNUHgz7EAlt1o',
    'id': '89XPBelvrZk',
    'statistics': {'

In [20]:
parsed_stats = parse_statistics(video_stats)
parsed_stats

{'views': [20060174,
  6894,
  1514644,
  31935326,
  11244,
  305157853,
  17324823,
  61027,
  154893638,
  130441139,
  7452990,
  395981,
  49300698,
  3446749,
  6968336,
  8300757,
  298484,
  59891630,
  98828,
  4367578,
  44769,
  1707917,
  3078089,
  2898,
  30124217,
  393050,
  7328,
  201901,
  15106539,
  1367973,
  23004292,
  442295,
  1436916,
  269292,
  120361151,
  146573,
  6892776,
  12105975,
  18016,
  226098,
  59229,
  251684,
  578621,
  109587,
  663211,
  138219,
  576071,
  1197289,
  1415157,
  684568,
  59229,
  30296,
  4389685,
  3796800,
  52683,
  95109,
  133883,
  46953,
  6875210,
  29476,
  56583,
  39889,
  33476,
  27162442,
  53237095,
  47357,
  6182278,
  2914684,
  12414544,
  26916894,
  6299902,
  3589347,
  3929082,
  793881,
  4809721,
  1661886,
  9508047,
  54514915,
  16801462,
  27499140,
  163233,
  24778,
  417371,
  50383,
  70332,
  2649766,
  665549,
  153024,
  2936456,
  116105,
  1426284,
  746721,
  513117,
  116372,
  941

In [21]:
video_df = pd.DataFrame(data = parsed_stats)
video_df.head()

,views,likes,dislikes,comments
0,20060174,116457,7690,10298
1,6894,173,3,0
2,1514644,21117,243,0
3,31935326,308118,12077,10827
4,11244,387,9,0


In [22]:
target = determine_target(video_df['views'])
target

[1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1]

In [16]:
video_df['target'] = target
#video_df.to_csv('video_data1.csv', index=False)
video_df.head()

,views,likes,dislikes,comments,target
0,20060174,116457,7690,10298,1
1,6894,172,3,0,0
2,1514572,21117,243,0,1
3,31935326,308118,12077,10827,1
4,10885,386,9,0,0
